### 필수 라이브러리 설치
- pip install celery redis elasticsearch
  - Celery: 비동기 작업 큐
  - Redis: 메시지 브로커(예시)
  - elasticsearch: 파이썬용 ElasticSearch 클라이언트

- 워커 실행 방법
  - celery -A tasks worker --loglevel=info


#### ElasticSearch 클라이언트 예제

In [1]:
from elasticsearch import Elasticsearch

# ElasticSearch 인스턴스 연결
es = Elasticsearch('localhost:9200')

# 데이터 색인(저장)
doc = {
    'user': 'kim',
    'message': 'hello elasticsearch',
}
es.index(index="msg", doc_type="store", body=doc)

# 데이터 검색
result = es.search(index="msg", query={"match": {"user": "kim"}})
print(result['hits']['hits'])


ValueError: URL must include a 'scheme', 'host', and 'port' component (ie 'https://localhost:9200')

#### Celery 기본 예제

In [2]:
from celery import Celery

# Celery 앱 생성
app = Celery('tasks', broker='redis://localhost:6379/0')

@app.task
def add(x, y):
    return x + y

# 비동기 작업 실행
result = add.delay(10, 20)
print(result.get())


OperationalError: Error 10061 connecting to localhost:6379. 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다.

#### ElasticSearch와 Celery 연동 예제

In [3]:
# tasks.py
from celery import Celery
from elasticsearch import Elasticsearch

app = Celery('tasks', broker='redis://localhost:6379/0')

@app.task
def index_document(index, doc_type, body):
    es = Elasticsearch('localhost:9200')
    res = es.index(index=index, doc_type=doc_type, body=body)
    return res

# 사용 예시
doc = {'user': 'lee', 'message': 'async with celery'}
result = index_document.delay('msg', 'store', doc)
print(result.get())


OperationalError: Error 10061 connecting to localhost:6379. 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다.